# Autoencoders

## Learning Outcomes
After doing the exercises below, you will be able to:
- Apply AEs to the image compression task.
- Define and train simple autoencoder models using pytorch.
- Evaluate the impact of different architecture choices.
- Visualize loss functions and image reconstructions.
- Benchmark results and compare them to common baselines.
- Argue about tranferability of latent representations in autoencoders.

## Library Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import random

import numpy as np
import torch
from torch.utils.data import DataLoader
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## The MNIST dataset
The MNIST dataset consists of **70,000 grayscale images** of handwritten digits (0 to 9). Each image is a **28x28 pixel** grayscale image. The training set contains 60,000 images while the test set contains 10,000 images. The grayscale values are integers ranging from 0 (black) to 255 (white). However, typicially the data is normalized to range between 0 and 1 for deep learning models.

The MNIST dataset can be downloaded from the **torchvision** library, using the following code:

In [3]:
# Load the MNIST dataset
transform = torchvision.transforms.ToTensor()

train_dataset = torchvision.datasets.MNIST(root='../../data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='../../data', train=False, download=True, transform=transform)

In [ ]:
# Plot 9 random images from the MNIST train dataset
def plot_images(img_dataset):
    """Plots 9 random images from the given dataset

    Args:
        img_dataset (torch.utils.data.Dataset): The dataset to plot images from
    """
    # Set up a 3x3 grid for plotting 9 random images
    figure = plt.figure(figsize=(8, 8))
    cols, rows = 3, 3
    
    # Plot 9 random images from the img_dataset
    for i in range(1, cols * rows + 1):
        random_idx = torch.randint(len(img_dataset), size=(1,)).item()  # Pick a random index
        img, label = img_dataset[random_idx]  # Get the image and its label
        figure.add_subplot(rows, cols, i)  # Add subplot
        plt.title(f'Label: {label}')  # Set title to show the label
        plt.axis("off")  # Turn off axis
        plt.imshow(img.squeeze(), cmap="gray")  # Plot image
    
    plt.show()  # Display the figure

plot_images(train_dataset)

**Exercise.** The models that we will be using in this notebook will mainly consist of fully connected layers and activations (no convolutions). Prepare the data for use with such models (complete all the `TODO` flags below):
1. Flatten the data.
2. Normalize pixel values to the interval $[0, 1]$.

In [ ]:
# Flatten the images
train_dataset = train_dataset.data.view( , ).float() # TODO: Complete with the appropriate dimensions
train_dataset = # TODO: Normalize pixel values.

# TODO: Repeat the same operations for the test dataset

In [6]:
# %load solutions/prepare_data.py
# Flatten the images and normalize the pixel values
train_dataset = train_dataset.data.view(-1, 28*28).float()
train_dataset = train_dataset / 255.0
test_dataset = test_dataset.data.view(-1, 28*28).float()
test_dataset = test_dataset / 255.0

## I. A couple of examples
We begin with a couple of examples that will prove useful in the sequel:
1. How to train PCA on MNIST using `scikit-learn`.
2. How to train a simple AE on MNIST using `Pytorch`.
Don't hesitate to refer back to these examples if needed.

### I.1. **Example.** training and evaluating PCA
In order to train and evaluate a PCA model, we use the PCA class provided by the `scikit-learn` dataset.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

# Instantiate a PCA model with the desired number of principal components
pca = PCA(n_components=10)

# Fit the PCA model to the training data
pca.fit(train_dataset)

# Evaluate the PCA model on the training and test data
pca_reconstruction_train = pca.inverse_transform(pca.transform(train_dataset))
pca_reconstruction_test = pca.inverse_transform(pca.transform(test_dataset))
pca_mse_train = mean_squared_error(train_dataset, pca_reconstruction_train)
pca_mse_test = mean_squared_error(test_dataset, pca_reconstruction_test)

# Print the PCA reconstruction MSE on the training and test data
print(f"PCA Reconstruction MSE on the train set: {pca_mse_train:.6f}")
print(f"PCA Reconstruction MSE on the test set: {pca_mse_test:.6f}")

The following code allows us to plot six random images from the MNIST test dataset along with their reconstructions using the PCA model:

In [ ]:
def image_comparison(original_images, reconstructions, n_images=6):
    """Plots the original images and its reconstructions for comparison

    Args:
        original_image (torch.Tensor): The original images
        reconstructions (torch.Tensor): Reconstruction of the original images
    """
    fig, ax = plt.subplots(2, n_images, figsize=(20, 7))
    
    for i in range(n_images):
        # Plot original images
        ax[0, i].imshow(original_images[i], cmap='gray')
        ax[0, i].axis('off')
        ax[0, 0].set_title('Original')

        # Reconstructed images
        ax[1, i].imshow(reconstructions[i], cmap='gray')
        ax[1, i].axis('off')
        ax[1, 0].set_title('Recomstruction')
    
    plt.tight_layout()
    plt.show()

# Select 6 random images from the test dataset
random_images = test_dataset[torch.randint(len(test_dataset), size=(6,))]

# Get the PCA reconstructions of the selected images
pca_reconstruction = pca.inverse_transform(pca.transform(random_images))

# Reshape the images for plotting
random_images = random_images.reshape(-1, 28, 28)
pca_reconstruction = pca_reconstruction.reshape(-1, 28, 28)

# Plot the original images and their PCA reconstructions
image_comparison(random_images, pca_reconstruction)

### I.2. **Example.** Training and evaluating an AE

In order to train and evaluate an AE model, we will use the `pytorch` library. For this example we will train a simple autoencoder with one fully connected layer plus activation in the encoder/decoder, and a free hidden dimension parameter. The steps we follow are:
1. We create an `Autoencoder` class with the desired architecture and a free hidden dimension parameter.
2. We prepare the data loaders to train the AE model using minibatch gradient descent.
3. We instatiate a model of the `Autoencoder` class with the desired hidden dimension parameter.
4. We define the loss and optimizer that will be used to train the AE.
5. We perform the actual training.
6. We evaluate the results.

#### I.2.1. Create the `Autoencoder` class
In `pytorch`, a neural network model is defined as a class that inherits from `nn.Module`. The class must have an `__init__` method that defines the layers of the network and a `forward` method that defines how input data is passed through the network. The `forward` method should return the output of the network. In this case, the `Autoencoder` class has an encoder and a decoder. Fully connected layers are called `Linear` in `Pytorch`.

In [14]:
# Create the Autoencoder class

class Autoencoder(nn.Module):
    def __init__(self, hidden_dim):
        super(Autoencoder, self).__init__()
        
        # Encoder: Linear layer with ReLU activation
        self.encoder = nn.Sequential(
            nn.Linear(28*28, hidden_dim),
            nn.ReLU()
        )

        # Decoder: Linear layer with Sigmoid activation
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, 28*28),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Encode the input
        x = self.encoder(x)
        # Decode the latent representation
        x = self.decoder(x)
        return x

#### I.2.2. Prepare the data loaders
Data loaders are used to iterate through data in batches.

In [15]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

#### I.2.3. Instantiate the model
We instantiate the model with a hidden dimension of 10. The `.to(device)` method moves the model to the GPU if available


In [16]:
hidden_dim = 10

simple_ae = Autoencoder(hidden_dim=hidden_dim).to(device)

#### I.2.4. Define loss and optimizer
Common loss functions can be access through pytorch's `.nn` module, whereas common optimizers are found in pytorch's `.optim` module. We have already imported both, as `nn` and `optim` respectively.

In [17]:
criterion = nn.MSELoss() # Mean Squared Error loss
optimizer = optim.Adam(simple_ae.parameters(), lr=1e-3) # Adam optimizer

#### I.2.5. Train the AE
Before each gradient descent iteration, it is extremely important to reset the gradients of the weights to 0 using the method `.zero_grad()`, otherwise, the new gradients are accumulated to those of previous iterations.

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    simple_ae.train()
    running_loss = 0.0
    for images in train_loader:
        # Move images to device
        images = images.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = simple_ae(images)
        
        # Compute loss
        loss = criterion(outputs, images)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.6f}")

#### I.2.6. Evaluate the results

We evaluate the results both by computing the test loss and by visualizing the reconstructed images against the original ones, as for PCA.

In [ ]:
simple_ae.eval()
test_loss = 0.0
with torch.no_grad():
    for images in test_loader:
        # Move images to the device
        images = images.to(device)
        
        # Forward pass
        outputs = simple_ae(images)
        loss = criterion(outputs, images)
        test_loss += loss.item() * images.size(0)

    test_loss /= len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.6f}")

In [ ]:
# Select a bath from the test loader
images = next(iter(test_loader))
images = images.to(device)

# Get the AE reconstructions of the selected images
ae_reconstruction = simple_ae(images)

# Reshape the images for plotting
random_images = images.cpu().numpy().reshape(-1, 28, 28)
ae_reconstruction = ae_reconstruction.cpu().detach().numpy().reshape(-1, 28, 28)

# Plot the original images and their PCA reconstructions
image_comparison(random_images, ae_reconstruction)

## II. Autoencoders vs PCA

The objective of this section is to review the following concepts, seen during the lesson:
> 1. A linear autoencoder learnt on the reconstruction loss is equivalent to PCA.
> 2. An autoencoder with nonlinear activations is more powerful than PCA.


### II.1. Linear AE vs PCA
In order to compare linear AEs and PCA, we will perform a benchmarking experiment. We will compare the PCA model with $p$ principal components with an undercomplete autoencoder with $p$ hidden dimensions, for different values of $p$. As a matter of fact, we will be using multiple autoencoder architectures, the hidden dimension will always be $p$, but the width and depth of the autoencoders may vary.

#### II.1.1. PCA training
Write a function that trains PCA with $p$ principal components by completing the `TODO` flag below.

In [99]:
def train_pca(p):
    """Trains a PCA model with p principal components

    Args:
        p (int): Number of principal components
    
    Returns:
        PCA: Trained PCA model
    """

    # TODO: Complete the function according to the docstring

In [100]:
# %load solutions/pca.py

#### II.1.2. Linear AE training
Write a function to train a linear autoencoder with the given hidden dimensions, width and depth by completing the `TODO` flags below.

In [102]:
class LinearAutoencoder(nn.Module):
    def __init__(self, hidden_dim, width, depth):
        super(LinearAutoencoder, self).__init__()
        
        # Intialize the encoder and decoder as empty Sequential models
        # We will add layers dynamically to these Sequential models according to the depth
        encoder = nn.Sequential()
        decoder = nn.Sequential()

        if depth < 1:
            raise ValueError("Depth must be at least 1") # TODO: Raise a ValueError

        if depth == 1:
            # In this case, we ignore the width parameter
            # and add a single linear layer to the encoder and decoder
            # with the appropriate dimensions.
            encoder.add_module("linear_1", nn.Linear(28*28, hidden_dim))
            decoder.add_module("linear_1", nn.Linear(hidden_dim, 28*28))

        else:
            # TODO: Add the first linear layer to the encoder
            for i in range(2, depth):
                # TODO: Add intermediate linear layers to the encoder
            # TODO: Add the last linear layer to the encoder

            # TODO: Add all necessary layers to the decoder

        # Assign the encoder and decoder to self so that they are recognized as part of the model
        self.encoder = encoder
        self.decoder = decoder

    # TODO: Implement the forward method

In [ ]:
# %load solutions/linear_ae.py

In [103]:
def train_autoencoder(hidden_dim, width, depth, epochs=5):
    # Instantiate the LinearAutoencoder model
    model = LinearAutoencoder(hidden_dim=hidden_dim, width=width, depth=depth).to(device)

    # Define the criterion and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    # Train the model
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images in train_loader:
            # Move images to device
            images = images.to(device)
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(images)
            # Compute loss
            loss = criterion(outputs, images)
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            # Update running loss
            running_loss += loss.item() * images.size(0)

        # Compute and print the epoch loss
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.6f}")

    return model, epoch_loss

#### II.1.3. Running the benchmark
Complete all the `TODO` flags below and perform the benchmark. Feel free to change the arrays `hidden_dims`, `ae_widths` and `ae_depths`.

In [104]:
def run_benchmark(hidden_dims, ae_widths, ae_depths):
    """Runs a benchmark on the given hidden dimensions, autoencoder widths, and autoencoder depths

    Args:
        hidden_dims (list): A list of hidden dimensions / principal components to test
        ae_widths (list): A list of autoencoder widths to test
        ae_depths (list): A list of autoencoder depths to test

    Returns:
        dict: Dictionary containing the PCA training loss
        dict: Dictionary containing the PCA test loss
        dict: Dictionary containing the autoencoder training loss
        dict: Dictionary containing the autoencoder test loss
    """
    
    # Initialize dictionaries to store results
    pca_train_loss = {}
    pca_test_loss = {}
    ae_train_loss = {}
    ae_test_loss = {}

    # Iterate through the hidden dimensions
    for p in hidden_dims:
        print(f"Training PCA model with {p} hidden dims")

        # TODO: Train a PCA model with p hidden dimensions

        # TODO: Build reconstruction of train/test data using the PCA model

        # Calculate the train / test MSE of the PCA model
        mse_train_pca =  # TODO: Evaluate the PCA model on train data
        mse_test_pca =  # TODO: Evaluate the PCA model on test data
        print(f"Train MSE: {mse_train_pca}, Test MSE: {mse_test_pca}")
        
        # Store the results in the dictionaries
        pca_train_loss[p] = mse_train_pca
        pca_test_loss[p] = mse_test_pca

        # Iterate through the autoencoder widths and depths
        for width in ae_widths:
            for depth in ae_depths:
                print(f"Training autoencoder with {p} hidden dims, width={width}, depth={depth}")

                # TODO: Initialize and train the autoencoder model

                # Build reconstruction of test data using AE and compute test loss
                model.eval()
                test_loss = 0.0
                with torch.no_grad():
                    for images in test_loader:
                        # TODO: Move images to device
                        # TODO: Forward pass on test images
                        loss = # TODO: Compute loss
                        test_loss += loss.item() * images.size(0)
                    
                    test_loss /= len(test_loader.dataset)

                # Store the results in the dictionaries
                ae_train_loss[(p, width, depth)] = # TODO: Store the train loss
                ae_test_loss[(p, width, depth)] = # TODO: Store the test loss
                print(f"Train MSE: {mse_train_ae}, Test MSE: {mse_test_ae}")

    return pca_train_loss, pca_test_loss, ae_train_loss, ae_test_loss

In [ ]:
# %load solutions/pca_vs_linear_ae_benchmark.py

In [ ]:
hidden_dims = [1, 2, 4, 8, 16]
ae_widths = [64] # We will only test one width, otherwise it takes too long...
ae_depths = [1, 2, 3]

# TODO: Run the benchmark

In [ ]:
# %load solutions/run_benchmark.py

#### II.1.4. Visualize the results
Compare the losses of the different models by plotting them into two similar graphs, one for the training MSE and another one for the test MSE:
- The $x$-axis should contain the different values $p$ for the hidden dimensions / principal components (in logarithmic scale).
- The $y$-axis should contain the corresponding MSE values.
- There should be one curve per model, with its own color and label.

In [ ]:
from collections import defaultdict

def plot_losses(pca_train_loss, pca_test_loss, ae_train_loss, ae_test_loss):
    """Plots the training and test losses for the PCA and autoencoder models.
    Produces two side-to-side plots for the train losses on the left 
    and the test losses on the right.

    Args:
        pca_train_loss (dict): Dictionary of PCA training losses
        pca_test_loss (dict): Dictionary of PCA test losses
        ae_train_loss (dict): Dictionary of autoencoder training losses (keys are tuples of (hidden_dim, width, depth))
        ae_test_loss (dict): Dictionary of autoencoder test losses (keys are tuples of (hidden_dim, width, depth))
    """
    # Create dictionaries to group the autoencoder losses by width and depth
    ae_train_grouped = defaultdict(list)
    ae_test_grouped = defaultdict(list)

    # Group losses by (width, depth) to have one curve per autoencoder model
    for (hidden_dim, width, depth), loss in ae_train_loss.items():
        ae_train_grouped[(width, depth)].append((hidden_dim, loss))

    for (hidden_dim, width, depth), loss in ae_test_loss.items():
        ae_test_grouped[(width, depth)].append((hidden_dim, loss))

    # Sort the losses by hidden_dim for each autoencoder model
    for key in ae_train_grouped:
        ae_train_grouped[key] = sorted(ae_train_grouped[key], key=lambda x: x[0])
    for key in ae_test_grouped:
        ae_test_grouped[key] = sorted(ae_test_grouped[key], key=lambda x: x[0])

    # Create a figure with two subplots, side-by-side
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Plot PCA losses
    ax[0].plot(list(pca_train_loss.keys()), list(pca_train_loss.values()), label='PCA Train Loss', marker='o')
    ax[1].plot(list(pca_test_loss.keys()), list(pca_test_loss.values()), label='PCA Test Loss', marker='o')

    # Plot Autoencoder losses grouped by (width, depth)
    for (width, depth), values in ae_train_grouped.items():
        hidden_dims = [v[0] for v in values]
        losses = [v[1] for v in values]
        ax[0].plot(hidden_dims, losses, label=f'AE Train Loss (width={width}, depth={depth})', marker='o')

    for (width, depth), values in ae_test_grouped.items():
        hidden_dims = [v[0] for v in values]
        losses = [v[1] for v in values]
        ax[1].plot(hidden_dims, losses, label=f'AE Test Loss (width={width}, depth={depth})', marker='o')

    # Set titles and labels
    ax[0].set_title('Training Losses')
    ax[0].set_xlabel('Hidden Dimension')
    ax[0].set_ylabel('Loss')
    ax[0].legend()
    ax[0].grid(True)

    ax[1].set_title('Test Losses')
    ax[1].set_xlabel('Hidden Dimension')
    ax[1].set_ylabel('Loss')
    ax[1].legend()
    ax[1].grid(True)

    # Adjust layout for better spacing
    plt.tight_layout()
    plt.show()

plot_losses(pca_train_loss, pca_test_loss, ae_train_loss, ae_test_loss)

#### II.1.5. Conclusion
Think about the following questions, do not hesitate to call us (the teaching staff) for a brief discussion about them !
- Can we see any difference between a linear AE and PCA?
- Does increasing depth/width have an impact on the performance of the linear AE? Why?
- For the task of data compression, if you had to choose between PCA with $p$ principal components, or a linear AE with $p$ hidden dimensions, which model would you choose? What are the reasons behind your choice?

### II.2. Nonlinear AEs vs PCA
In this section we will compare some of the non-linear AE models seen during the lesson to our PCA baseline. Indeed, if we are going to adopt an AE model for a given task, we will need to make sure that its performance is better than that of PCA, otherwise there is no point on adopting a more complex architecture.
The models we will evaluate are the following ones:
> - Undercomplete AE
> - Denoising AE

To keep it simple, we will omit the implementations of the sparse and contractive autoencoders.
Once again, we will perform a benchmarking experiment in order to evaluate the different AE models and compare them agains PCA and against each other.

Throughout this section, we will fix: 
- An array of hidden dimensions to test
- The depth of the AE architectures to be defined. 
- The width of the AE architectures to be defined.
- The batch size.
- The number of training epochs.

In [140]:
HIDDEN_DIMS = [1, 2, 4, 8, 16]
INPUT_DIM = 28*28
WIDTH = 128
DEPTH = 2
BATCH_SIZE = 64
EPOCHS = 5

For each of the models that we are studying, we will need to:
- Define the architecture.
- Train the model with different hyper-parameters.
- Select the hyper-parameters corresponding to the best model.
- Evaluate the model.

As it is customary in Deep Learning practice, the hyper-parameter optimization is done with the help of a validation set, therefore, we will split the test dataset into a validation dataset and a proper test dataset, and prepare the corresponding data loaders.

In [108]:
# Split test data into validation and test sets
val_dataset, test_dataset = torch.utils.data.random_split(test_dataset, [5000, 5000])

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

**Exercice.** In order to perform the benchmark explained above, complete the following sections by addressing all the `TODO` flags in them:
1. Define a common architecture for both models.
2. Train and evaluate the undercomplete AE. Select the best model after hyper-parameter optimization if applies.
3. Train and evaluate the denoising AE. Select the best model after hyper-parameter optimization if applies.
7. Compare the performances (MSE loss) of the selected models (after hyper-parameter optimization) between each other and agains PCA.
8. Visualize the image reconstructions for both models and for PCA.

#### II.2.1. Architecture
**Question.** How many different architectures should we define ?

In [168]:
# TODO: Define a general autoencoder architecture with the fixed global parameters
# all activations are ReLU except the last one.

In [ ]:
# %load solutions/common_ae_architecture.py

#### II.2.2. Undercomplete AE
**Question.** Are there any hyper-parameters involved in the training loss of an undercomplete AE?

In [ ]:
# Train the undercomplete AE for the different hidden dimensions

# Store the models, along with the losses
ucae_models = {} # For visualization of image reconstructions
ucae_train_losses = {}
ucae_val_losses = {}
ucae_test_losses = {}

for p in HIDDEN_DIMS:
    print(f"Training undercomplete AE with {p} hidden dims")

    # TODO: Instantiate the model and move it to the device

    # TODO: Set the loss and optimizer

    for epoch in range(EPOCHS):
        # TODO: Set the model to training mode
        # TODO: Initialize running training and validation loss for the epoch

        # TODO: Training loop
    
        # Compute average training loss for the epoch
        train_loss = # TODO: Compute the average training loss

        # Validation loop
        undercomplete_ae.eval()  # Set model to evaluation mode
        with torch.no_grad():  # Disable gradient calculation for validation
            for images in val_loader:
                # Move images to device
                images = images.to(device)
                # Forward pass
                outputs = undercomplete_ae(images)
                # Compute loss
                loss = criterion(outputs, images)
                # Accumulate validation loss
                running_val_loss += loss.item() * images.size(0)

        # Compute average validation loss for the epoch
        val_loss = running_val_loss / len(val_loader.dataset)

        # Print training and validation statistics for the epoch
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    # Save the trained model and losses
    ucae_models[p] = undercomplete_ae
    ucae_train_losses[p] = train_loss
    ucae_val_losses[p] = val_loss

    # TODO: Evaluate the model on the test set
    
    # Compute average test loss
    test_loss = # TODO: Compute the average test loss
    ucae_test_losses[p] = test_loss

In [ ]:
# %load solutions/train_ucae.py

#### II.2.3. Denoising AE
**Question.** Are there any new hyper-parameters involved in the training of the Denoising AE?

In [ ]:
# Train the denoising AE with different noise levels

# Set the hyper-parameters
noise_levels = [0.003, 0.01, 0.03, 0.1]  # Noise levels to test

# Store the models, along with the losses
denoising_models = {}
denoising_train_losses = {}
denoising_val_losses = {}
denoising_test_losses = {}

for p in HIDDEN_DIMS:
    for noise_level in noise_levels:
        print(f"Training denoising AE with {p} hidden dims and noise level {noise_level}")
        # TODO: Instantiate the model and move it to the device

        # TODO: Set the loss and optimizer

        for epoch in range(EPOCHS):
            # TODO: Set the model to training mode
            # TODO: Initialize running training and validation loss for the epoch

            # TODO: Training loop keeping in mind to:
            # generate noisy images with the given noise level
            # clip pixel values to [0, 1] after adding noise
            # do the forward pass on the noisy images
            # compute the loss by comparing the outputs with the ORIGINAL images
            
            # TODO: Compute average training loss for the epoch

            # TODO: Validation loop keeping in mind to:
            # follow the same procedure as for the training loop.

            # TODO: Compute average validation loss for the epoch

            # TODO: Print training and validation statistics for the epoch

        # TODO: Save model and losses
        denoising_models[(p, noise_level)] = # TODO: Save the model
        denoising_train_losses[(p, noise_level)] = # TODO: Save the training loss
        denoising_val_losses[(p, noise_level)] = # TODO: Save the validation loss

        # TODO: Evaluate the model on the test set
        
        # TODO: Compute and save the average test loss
        test_loss = # TODO: Compute the average test loss
        denoising_test_losses[(p, noise_level)] = test_loss
        

In [ ]:
# %load solutions/train_denoising_ae.py

#### II.2.4. Loss plots

In [ ]:
# Train PCA

pca_train_losses = {}
pca_val_losses = {}
pca_test_losses = {}

for p in HIDDEN_DIMS:
    print(f"Training PCA model with {p} hidden dims")

    # Train the PCA model
    pca = train_pca(p)

    # Evaluate the PCA model on the training and validation data
    pca_reconstruction_train = pca.inverse_transform(pca.transform(train_dataset))
    pca_reconstruction_val = pca.inverse_transform(pca.transform(val_dataset))
    pca_reconstruction_test = pca.inverse_transform(pca.transform(test_dataset))

    mse_train_pca = mean_squared_error(train_dataset, pca_reconstruction_train)
    mse_val_pca = mean_squared_error(val_dataset, pca_reconstruction_val)
    mse_test_pca = mean_squared_error(test_dataset, pca_reconstruction_test)
    print(f"Train MSE: {mse_train_pca}, Validation MSE: {mse_val_pca}")

    pca_train_losses[p] = mse_train_pca
    pca_val_losses[p] = mse_val_pca
    pca_test_losses[p] = mse_test_pca

In [179]:
# Create dictionaries to group the denoising AE losses by noise level
denoising_train_grouped = defaultdict(list)
denoising_val_grouped = defaultdict(list)

# Group losses by noise level to have one curve per denoising model
for (hidden_dim, noise_level), loss in denoising_train_losses.items():
    denoising_train_grouped[noise_level].append((hidden_dim, loss))
for (hidden_dim, noise_level), loss in denoising_val_losses.items():
    denoising_val_grouped[noise_level].append((hidden_dim, loss))

# Sort the losses by hidden_dim for each denoising model
for key in denoising_train_grouped:
    denoising_train_grouped[key] = sorted(denoising_train_grouped[key], key=lambda x: x[0])
for key in denoising_val_grouped:
    denoising_val_grouped[key] = sorted(denoising_val_grouped[key], key=lambda x: x[0])

In [ ]:
# Create a figure with two subplots, side-by-side
# one for training losse curves and the other for validation loss curves

fig, ax = plt.subplots(1, 2, figsize=(14, 4))
#fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot PCA losses
ax[0].plot(list(pca_train_losses.keys()), list(pca_train_losses.values()), label='PCA', marker='o')
ax[1].plot(list(pca_val_losses.keys()), list(pca_val_losses.values()), label='PCA', marker='o')
#ax[2].plot(list(pca_test_loss.keys()), list(pca_test_loss.values()), label='PCA', marker='o')

# Plot undercomplete AE losses
ax[0].plot(list(ucae_train_losses.keys()), list(ucae_train_losses.values()), label='UCAE', marker='o')
ax[1].plot(list(ucae_val_losses.keys()), list(ucae_val_losses.values()), label='UCAE', marker='o')
#ax[2].plot(list(ucae_test_losses.keys()), list(ucae_test_losses.values()), label='UCAE', marker='o')

# Plot denoising AE losses
for noise_level, values in denoising_train_grouped.items():
    hidden_dims = [v[0] for v in values]
    losses = [v[1] for v in values]
    ax[0].plot(hidden_dims, losses, label=f'Denoising AE (noise={noise_level})', marker='o')

for noise_level, values in denoising_val_grouped.items():
    hidden_dims = [v[0] for v in values]
    losses = [v[1] for v in values]
    ax[1].plot(hidden_dims, losses, label=f'Denoising AE (noise={noise_level})', marker='o')

# Set titles and labels
ax[0].set_title('Training Losses')
ax[0].set_xlabel('Hidden Dimension')
ax[0].set_ylabel('Loss')
ax[0].legend()
ax[0].grid(True)

ax[1].set_title('Validation Losses')
ax[1].set_xlabel('Hidden Dimension')
ax[1].set_ylabel('Loss')
ax[1].legend()
ax[1].grid(True)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

**Question.** What conclusion do you draw from these plots ? Has anything changes as a consequence of the addition of the non-linear activations ?

Since we are using a validation test, we can rely on it for choosing the best noise level in the denoising AE for each choice of the hidden parameter $p$. The code below computes the noise level that achieves best validation loss, and plots a curve of "best denoising models" against PCA and the undercomplete autoencoder. 

In [ ]:
# Evaluate the best models on the test set

# Dictionary to store the best test losses and their corresponding noise levels
best_test_losses = {}

# Iterate over hidden dimensions and select the best model based on validation loss
for p in HIDDEN_DIMS:
    # Find the noise level that has the minimum validation loss for the current hidden dimension
    best_noise_level = min(
        [noise_level for noise_level in noise_levels],
        key=lambda nl: denoising_val_losses[(p, nl)]
    )

    # Retrieve the best model, train loss, and validation loss for the selected noise level
    best_test_loss = denoising_test_losses[(p, best_noise_level)]
    best_train_loss = denoising_train_losses[(p, best_noise_level)]
    best_val_loss = denoising_val_losses[(p, best_noise_level)]
    best_test_losses[p] = best_test_loss

    # Print out the best models, their noise levels, and the corresponding losses
    print(f"Best model for hidden dimension {p}:")
    print(f"Noise level: {best_noise_level}")
    print(f"Train loss: {best_train_loss}")
    print(f"Validation loss: {best_val_loss}")
    print(f"Test loss: {best_test_loss}")

In [ ]:
# Plot test losses for PCA, UCAE, and best denoising AEs

fig, ax = plt.subplots(1, 1, figsize=(7, 4))

# Plot PCA test loss
ax.plot(list(pca_test_losses.keys()), list(pca_test_losses.values()), label='PCA', marker='o')

# Plot undercomplete AE test loss
ax.plot(list(ucae_test_losses.keys()), list(ucae_test_losses.values()), label='UCAE', marker='o')

# Plot best denoising AE test loss
ax.plot(list(best_test_losses.keys()), list(best_test_losses.values()), label='Best Denoising AE per hidden dim', marker='o')

# Set titles and labels
ax.set_title('Test Losses')
ax.set_xlabel('Hidden Dimension')
ax.set_ylabel('Loss')
ax.legend()
ax.grid(True)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

#### II.2.5. Visualization of reconstructions
Finally, we are going to compare the reconstructions of the original images by the three models, for the hidden dimension $p=8$.

In [ ]:
# Visualize the image reconstructions for 
# PCA, UCAE, and the best denoising AE model for hidden_dim=8

# Get the best denoising AE model for hidden_dim=8
denoising_model = denoising_models[(8, 0.03)]

# Get a batch of test images
images = next(iter(test_loader))
images = images.to(device)

# Generate noisy images and clip pixel values to [0, 1]
noisy_images = images + 0.03 * torch.randn_like(images)
noisy_images = torch.clamp(noisy_images, 0, 1)

# Reconstruct images using the three models
pca_reconstruction = pca.inverse_transform(pca.transform(images.view(images.size(0), -1).cpu().numpy()))
ucae_reconstruction = ucae_models[8](images.view(images.size(0), -1)).cpu().detach().numpy()
denoising_reconstruction = denoising_model(noisy_images.view(noisy_images.size(0), -1)).cpu().detach().numpy()

# Reshape images for plotting to (batch_size, 28, 28)
images = images.cpu().numpy().reshape(-1, 28, 28)
noisy_images = noisy_images.cpu().numpy().reshape(-1, 28, 28)
pca_reconstruction = pca_reconstruction.reshape(-1, 28, 28)
ucae_reconstruction = ucae_reconstruction.reshape(-1, 28, 28)
denoising_reconstruction = denoising_reconstruction.reshape(-1, 28, 28)


# Plot the original, PCA, UCAE, noisy, and denoising AE reconstructed images
fig, ax = plt.subplots(5, 6, figsize=(20, 10))

for i in range(6):
    # Original images
    ax[0, i].imshow(images[i], cmap='gray')
    ax[0, i].axis('off')
    ax[0, 0].set_title('Original')

    # PCA reconstructed images
    ax[1, i].imshow(pca_reconstruction[i], cmap='gray')
    ax[1, i].axis('off')
    ax[1, 0].set_title('PCA')

    # UCAE reconstructed images
    ax[2, i].imshow(ucae_reconstruction[i], cmap='gray')
    ax[2, i].axis('off')
    ax[2, 0].set_title('UCAE')

    # Noisy images
    ax[3, i].imshow(noisy_images[i], cmap='gray')
    ax[3, i].axis('off')
    ax[3, 0].set_title('Noisy')

    # Denoising AE reconstructed images
    ax[4, i].imshow(denoising_reconstruction[i], cmap='gray')
    ax[4, i].axis('off')
    ax[4, 0].set_title('Denoising AE')

plt.tight_layout()
plt.show()

## III.Trasferable Representations?
The objective of this section is to answer to the following question:
- Are the latent representations learnt by an autoencoder transferable to other tasks ?

To that end, we will train an autoencoder on the MNIST dataset, and use the latent representations along with the image labels to train a classifier, and evaluate its performance.

In [28]:
# Load the MNIST dataset
from torchvision import datasets, transforms

transform = transforms.ToTensor()
batch_size = 64

# Load datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define and train the model
encoding_dim = 10  # Undercomplete dimension
autoencoder = Autoencoder(encoding_dim).to(device)

criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)

num_epochs = 10

for epoch in range(num_epochs):
    autoencoder.train()
    running_loss = 0.0
    for images, _ in train_loader:
        # Flatten images
        images = images.to(device).view(-1, 28*28)
        # Forward pass
        outputs = autoencoder(images)
        loss = criterion(outputs, images.view(-1, 28*28))
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

In [36]:
# Extract latent representations
def get_latent_features(model, dataloader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for images, lbls in dataloader:
            images = images.to(device).view(-1, 28*28)
            latent = model.encoder(images)
            features.append(latent.cpu())
            labels.append(lbls)
    features = torch.cat(features)
    labels = torch.cat(labels)
    return features, labels

In [41]:
# Prepare data loaders for classification
from torch.utils.data import TensorDataset

train_features, train_labels = get_latent_features(autoencoder, train_loader)
test_features, test_labels = get_latent_features(autoencoder, test_loader)

train_dataset_cls = TensorDataset(train_features, train_labels)
test_dataset_cls = TensorDataset(test_features, test_labels)

train_loader_cls = DataLoader(dataset=train_dataset_cls, batch_size=batch_size, shuffle=True)
test_loader_cls = DataLoader(dataset=test_dataset_cls, batch_size=batch_size, shuffle=False)

In [42]:
# Define the classifier architecture

class Classifier(nn.Module):
    def __init__(self, input_dim=encoding_dim, num_classes=10):
        super(Classifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(True),
            nn.Linear(64, 32),
            nn.ReLU(True),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
# Initialize and train the classifier

classifier = Classifier().to(device)

criterion_cls = nn.CrossEntropyLoss()
optimizer_cls = optim.Adam(classifier.parameters(), lr=1e-3)

num_epochs_cls = 20

for epoch in range(num_epochs_cls):
    classifier.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for features, labels in train_loader_cls:
        features = features.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = classifier(features)
        loss = criterion_cls(outputs, labels)
        # Backward and optimize
        optimizer_cls.zero_grad()
        loss.backward()
        optimizer_cls.step()
        running_loss += loss.item()
        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    avg_loss = running_loss / len(train_loader_cls)
    accuracy = 100 * correct / total

    # Compute the test loss and accuracy
    classifier.eval()
    train_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader_cls:
            features = features.to(device)
            labels = labels.to(device)
            outputs = classifier(features)
            loss = criterion_cls(outputs, labels)
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_loss = train_loss / len(test_loader_cls)
        test_accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs_cls}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

**Question.** Are the representations learnt by the autoencoder transferable to the classification task ?